In [ ]:
# default_exp core

# Tools to create macros for GATE simulations

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import math
def Ek(mass, momentum):
    '''Helpfull function that converts momentum to kinetic energy'''
    return math.sqrt(mass**2 + momentum**2) - mass

In [ ]:
#export
import subprocess
from os import path
from time import strftime

class MacroWriter:
    def __init__(self, macro_folder='mac', results_folder='results', logs_folder='logs', sensor_material='Silicon'):
        self.macro = macro_folder
        self.results = results_folder
        self.logs = logs_folder
        self.sensor_material = sensor_material
        
    def create_sensor(self, n=0, x_length=20, z_length=20, thickness=0.1, x_loc=0, y_loc=0, z_loc=0, 
                  system='scanner'):
    
        """Write a GATE macro file for creating a sensor

        In the current implementation snesor is a flat plane perpendicular to the beam 
        which is by default in y direction
        thickness dimension is in mm
        all other dimensions are in cm"""



        lines = f"""#sensor
        /gate/scanner/daughters/name sensor{n}
        /gate/{system}/daughters/insert box
        /gate/sensor{n}/geometry/setXLength      {x_length} cm
        /gate/sensor{n}/geometry/setYLength      {thickness} mm
        /gate/sensor{n}/geometry/setZLength      {z_length} cm
        /gate/sensor{n}/setMaterial {self.sensor_material}
        /gate/sensor{n}/placement/setTranslation {x_loc} {y_loc} {z_loc} cm
        /gate/sensor{n}/vis/setVisible 1
        /gate/sensor{n}/vis/setColor yellow
        /gate/systems/{system}/level1/attach sensor{n}
        /gate/sensor{n}/attachCrystalSD"""


        with open(f'{self.macro}/sensor{n}.mac', 'w') as file:
            file.write(lines)
            
            
    def create_phantom_layer(self, n=0, x_length=20, z_length=20, thickness=1, x_loc=0, y_loc=0, z_loc=0, material='Water'):
        '''
        Write a GATE macro for creating a phantom box.
        
        which in the current implementation is a flat box perpendicular to beam which is in y direction
        all dimensions are in cm
        '''
        lines = f"""
        #phatom box {n}
        /gate/world/daughters/name              box{n}
        /gate/world/daughters/insert            box
        /gate/box2/geometry/setXLength      {x_length} cm
        /gate/box2/geometry/setYLength      {thickness} cm
        /gate/box2/geometry/setZLength      {z_length} cm
        /gate/box2/placement/setTranslation {x_loc} {y_loc} {z_loc} cm
        /gate/box2/setMaterial {material}
        /gate/box2/vis/setVisible 1
        /gate/box2/vis/setColor blue
        #*************************************************************
        #  Physics (infrared divergence) cuts for the layer{n}
        #*************************************************************
        /gate/physics/Gamma/SetCutInRegion      box{n} 0.1 mm
        /gate/physics/Electron/SetCutInRegion   box{n} 0.1 mm
        /gate/physics/Positron/SetCutInRegion   box{n} 0.1 mm

        #*************************************************************
        # attached actor to the box{0}
        #***************************************************************
        /gate/actor/addActor PhaseSpaceActor         tracker{n}
        /gate/actor/tracker{n}/save                     tracker{n}.root
        /gate/actor/tracker{n}/attachTo                 box{n}
        /gate/actor/tracker{n}/enableNuclearFlag          true
        /gate/actor/tracker{n}/enableProductionProcess  false
        #/gate/actor/tracker{n}/useVolumeFrame           true
        /gate/actor/tracker{n}/storeOutgoingParticles true
        """
        with open(f'{self.macro}/phantom{n}.mac', 'w') as file:
                file.write(lines)
        
    def create_beam(self, energy=250, sigma_energy=1.0, position={'x':0, 'y':250, 'z':0}):
        lines = f'''
        #=====================================================
        # BEAMS
        #=====================================================
        /gate/source/addSource PBS PencilBeam
        /gate/source/PBS/setParticleType proton
        /gate/source/PBS/setEnergy {energy} MeV
        /gate/source/PBS/setSigmaEnergy {sigma_energy} MeV
        /gate/source/PBS/setPosition {position.x} {position.y} {position.z} mm
        /gate/source/PBS/setSigmaX 2 mm
        /gate/source/PBS/setSigmaY 2 mm
        /gate/source/PBS/setSigmaTheta 3 mrad
        /gate/source/PBS/setSigmaPhi 3 mrad
        /gate/source/PBS/setEllipseXThetaEmittance 15 mm*mrad
        #/gate/source/PBS/setEllipseXThetaRotationNorm negative
        /gate/source/PBS/setEllipseYPhiEmittance 15 mm*mrad
        /gate/source/PBS/setEllipseYPhiRotationNorm negative
        /gate/source/PBS/setRotationAxis 1 0 0
        /gate/source/PBS/setRotationAngle 90 deg
        #/gate/application/setTotalNumberOfPrimaries 10000
        '''
        with open(f'{self.macro}/beam{energy}.mac', 'w') as f:
            f.write(lines)
            
    def create_start_beams(n_primaries=10000):
        lines = f"""
        #=====================================================
        # START BEAMS
        #=====================================================

        # JamesRandom Ranlux64 MersenneTwister
        /gate/random/setEngineName MersenneTwister
        /gate/random/setEngineSeed 123456

        # /gate/random/verbose 1
        # /gate/source/verbose 0

        # to check Steplimiter
        #/tracking/verbose 1

        #/gate/application/noGlobalOutput
        /gate/application/setTotalNumberOfPrimaries {n_primaries}
        """
        with open(f'{self.macro}/start_beam{n_primaries}.mac', 'w') as f:
            f.write(lines)
            
    def create_macro_file(self, macro_dict):
        lines = f"""
        #=====================================================
        # GEOMETRY
        #=====================================================
        /control/execute {macro_folder}/{macro_dict["geometry"]}.mac    
        #=====================================================
        # PHYSICS
        #=====================================================
        /control/execute {macro_folder}/{macro_dict["physics"]}.mac
        #================================================================
        #     ACTORS
        #================================================================
        /control/execute {macro_folder}/{macro_dict["actors"]}.mac
        #=====================================================
        # INITIALISATION
        #=====================================================
        /control/execute {macro_folder}/{macro_dict["intialisation"]}.mac
        #=====================================================
        # BEAMS
        #=====================================================
        /control/execute {macro_folder}/{macro_dict["beam"]}.mac
        #=====================================================
        # START BEAMS
        #=====================================================
        /control/execute {macro_folder}/{macro_dict["start_beam"]}.mac
        #===============================================
        # OUTPUT SETTINGS
        #===============================================
        /gate/application/start
        exit
        """
        timestamp = strftime("%Y%b%d_%H%M")
        with open(f'{self.macro}/main_macro{timestamp}.mac', 'w') as f:
            f.write()

In [ ]:
my_macro = MacroWriter()

In [ ]:
#export
def run_macro(macroname, log_folder='logs'):
    log_file_name = f'gate_stdout_err_{strftime("%Y%b%d_%H%M")}.log'
    log_path = path.join(log_folder, log_file_name)
    with open(log_path,'a+') as f_stdout:
        subprocess.run(['Gate', macroname], stdout=f_stdout, stderr=subprocess.STDOUT)
    

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_macro_tools.ipynb.
Converted 01_analysis.ipynb.
Converted index.ipynb.


In [ ]:
ls

00_core.ipynb       MANIFEST.in         docs/               mac/
01_analysis.ipynb   Makefile            gate_simulation/    settings.ini
CONTRIBUTING.md     README.md           gate_stdout.log     setup.py
LICENSE             docker-compose.yml  index.ipynb         simdata.csv
